Общие шаги, которые необходимо выполнить:
1. Загрузка данных: Загрузим данные из предоставленного CSV файла.
2. Предварительная обработка данных: Выполним обработку данных, чтобы подготовить их для обучения модели (например, кодирование категориальных признаков, обработка пропущенных значений и т.д.).
3. Разделение данных на тренировочные и тестовые: Разделим данные на тренировочный и тестовый наборы, чтобы мы могли оценить производительность модели.
4. Обучение модели: Выберем подходящий алгоритм машинного обучения и обучим модель на тренировочных данных.
5. Сохранение модели: Сохраним обученную модель на диск, чтобы её можно было использовать без повторного обучения.
6. Реализация API: Создадим REST-API для работы с моделью (запрос прогноза, обучение, получение коэффициентов).
7. Оценка модели: Оценим качество модели на тестовых данных.

Давайте начнем с загрузки и анализа данных, а затем приступим к созданию модели.

In [8]:
import pandas as pd

# Load the dataset
file_path = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v1/data/learning_data_v1.csv'
data = pd.read_csv(file_path, low_memory=False)

# Display basic information about the dataset
data_info = data.info()
data_head = data.head()

data_info, data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74974 entries, 0 to 74973
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 74974 non-null  int64  
 1   type               74974 non-null  object 
 2   processing_method  74974 non-null  object 
 3   sku_id             74974 non-null  int64  
 4   op_pallet_pick     74974 non-null  float64
 5   op_cont_pick       74974 non-null  float64
 6   op_cont_deliver    74974 non-null  float64
 7   op_cont_aboard     74974 non-null  float64
 8   op_load_deliver    74974 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 5.1+ MB


(None,
        id type processing_method   sku_id  op_pallet_pick  op_cont_pick  \
 0  245673    3                 b  1033531        0.200000           0.0   
 1  245673    3                 b  1032755        0.200000           0.0   
 2  245673    3                 b  1033157        0.200000           0.0   
 3  245673    3                 b  1032922        0.200000           0.0   
 4  245717    3                 b  1032789        0.467742           0.0   
 
    op_cont_deliver  op_cont_aboard  op_load_deliver  
 0              0.0             0.0         0.800000  
 1              0.0             0.0         0.800000  
 2              0.0             0.0         0.800000  
 3              0.0             0.0         0.800000  
 4              0.0             0.0         0.532258  )

Данные успешно загружены. Вот основные моменты:
Признаки:
    id (идентификатор заказа)
    type (тип заказа)
    processing_method (метод обработки)
    sku_id (идентификатор товара)
Целевые переменные:
    op_pallet_pick (отбор паллет)
    op_cont_pick (отбор коробок)
    op_cont_deliver (довоз контейнеров)
    op_cont_aboard (комиссионирование)
    op_load_deliver (вытеснение)

Каждая запись представляет собой комбинацию признаков и целевых переменных, где целевые переменные показывают процент операций, требуемых для сборки заказа.

Далее проведем кодирование категориальных признаков и разделение данных на тренировочные и тестовые наборы. ​

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical features
label_encoder_type = LabelEncoder()
label_encoder_processing_method = LabelEncoder()

data['type_encoded'] = label_encoder_type.fit_transform(data['type'])
data['processing_method_encoded'] = label_encoder_processing_method.fit_transform(data['processing_method'])

# Features and target variables
X = data[['type_encoded', 'processing_method_encoded', 'sku_id']]
y = data[['op_pallet_pick', 'op_cont_pick', 'op_cont_deliver', 'op_cont_aboard', 'op_load_deliver']]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((59979, 3), (14995, 3), (59979, 5), (14995, 5))

Данные успешно разделены на тренировочный и тестовый наборы:
Тренировочный набор: 59979 записей
Тестовый набор: 14995 записей
Признаки (X): 3 столбца (type_encoded, processing_method_encoded, sku_id)
Целевые переменные (y): 5 столбцов (op_pallet_pick, op_cont_pick, op_cont_deliver, op_cont_aboard, op_load_deliver)

Теперь можно перейти к выбору и обучению модели. Для данной задачи хорошо подойдет модель мультивыходной регрессии, например, RandomForestRegressor или GradientBoostingRegressor, поскольку она может предсказывать несколько целевых переменных одновременно.

Начнем с обучения модели и оценим её качество.

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
model = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mse

array([0.00027233, 0.06422447, 0.00570605, 0.07814752, 0.00011397])

Модель обучена, и её производительность оценена с использованием среднеквадратичной ошибки (MSE) для каждой целевой переменной:
op_pallet_pick: 0.00027233
op_cont_pick: 0.06422447
op_cont_deliver: 0.00570605
op_cont_aboard: 0.07814752
op_load_deliver: 0.00011397
MSE показывает среднеквадратичное отклонение между предсказанными и реальными значениями. Чем меньше значение, тем лучше модель справляется с предсказанием.

Теперь сохраним обученную модель на диск для её последующего использования.

In [11]:
import joblib

# Save the trained model to a file
model_filename = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v1/model/modelorder_fulfillment_model.pkl'
joblib.dump(model, model_filename)

# Also save the label encoders to use for encoding during prediction
label_encoders_filename = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v1/model/modellabel_encoders.pkl'
joblib.dump({
    'type_encoder': label_encoder_type,
    'processing_method_encoder': label_encoder_processing_method
}, label_encoders_filename)

model_filename, label_encoders_filename

('/Users/sgzh1/projects/order-fulfillment-forecast-requires/v1/model/modelorder_fulfillment_model.pkl',
 '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v1/model/modellabel_encoders.pkl')